In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [2]:
from subprocess import check_output

In [3]:
from sklearn.model_selection import train_test_split

In [4]:
def read_data():
    df = pd.read_csv("train.csv",nrows=200000)
    print ("Shape of base training File = ", df.shape)
    # Remove missing values and duplicates from training data
    df.drop_duplicates(inplace=True)
    df.dropna(inplace=True)
    print("Shape of base training data after cleaning = ", df.shape)
    return df
df = read_data()

df_train, df_test = train_test_split(df, test_size = 0.02)
#print(df_train)
#df_train = df_train[0:100]
#df_test = df_test[0:100]
#print (df_train.head(2))
print (df_test.shape)
print(df_train.shape)
print(df_train)

('Shape of base training File = ', (200000, 6))
('Shape of base training data after cleaning = ', (199999, 6))
(4000, 6)
(195999, 6)
            id    qid1    qid2  \
140411  140411  219011   28072   
6931      6931   13565   13566   
115104  115104  187761  187762   
83685    83685  141615  141616   
42683    42683   76882   76883   
105446  105446  173884  173885   
17440    17440   33138   33139   
140302  140302  222982  222983   
31964    31964   58884   58885   
49789    49789   88574   88575   
160214  160214  249930   29858   
182003  182003   13206  125506   
161023  161023  251013  251014   
188904  188904  287546  287547   
7638      7638   14915   11046   
129067  129067  207423  207424   
90619    90619  152036  152037   
108571  108571  178358  178359   
157023  157023  245610  245611   
87630    87630  147553  147554   
75347    75347  128971  128972   
7779      7779   15185   15186   
25284    25284   47140   47141   
37403    37403   68070   68071   
98181    98181  1

In [5]:
from collections import Counter
import matplotlib.pyplot as plt
import operator

In [6]:

def eda(df):
    print ("Duplicate Count = %s , Non Duplicate Count = %s" 
           %(df.is_duplicate.value_counts()[1],df.is_duplicate.value_counts()[0]))
    
    question_ids_combined = df.qid1.tolist() + df.qid2.tolist()
    
    print ("Unique Questions = %s" %(len(np.unique(question_ids_combined))))
    
    question_ids_counter = Counter(question_ids_combined)
    sorted_question_ids_counter = sorted(question_ids_counter.items(), key=operator.itemgetter(1))
    question_appearing_more_than_once = [i for i in question_ids_counter.values() if i > 1]
    print ("Count of Quesitons appearing more than once = %s" %(len(question_appearing_more_than_once)))

In [7]:
eda(df_train)
print(df_train)

Duplicate Count = 72964 , Non Duplicate Count = 123035
Unique Questions = 296492
Count of Quesitons appearing more than once = 46708
            id    qid1    qid2  \
140411  140411  219011   28072   
6931      6931   13565   13566   
115104  115104  187761  187762   
83685    83685  141615  141616   
42683    42683   76882   76883   
105446  105446  173884  173885   
17440    17440   33138   33139   
140302  140302  222982  222983   
31964    31964   58884   58885   
49789    49789   88574   88575   
160214  160214  249930   29858   
182003  182003   13206  125506   
161023  161023  251013  251014   
188904  188904  287546  287547   
7638      7638   14915   11046   
129067  129067  207423  207424   
90619    90619  152036  152037   
108571  108571  178358  178359   
157023  157023  245610  245611   
87630    87630  147553  147554   
75347    75347  128971  128972   
7779      7779   15185   15186   
25284    25284   47140   47141   
37403    37403   68070   68071   
98181    98181  1

In [8]:
import re
import gensim
from gensim import corpora
from nltk.corpus import stopwords
from nltk.stem.porter import *

words = re.compile(r"\w+",re.I)
stopword = stopwords.words('english')
stemmer = PorterStemmer()

print(df_train)

            id    qid1    qid2  \
140411  140411  219011   28072   
6931      6931   13565   13566   
115104  115104  187761  187762   
83685    83685  141615  141616   
42683    42683   76882   76883   
105446  105446  173884  173885   
17440    17440   33138   33139   
140302  140302  222982  222983   
31964    31964   58884   58885   
49789    49789   88574   88575   
160214  160214  249930   29858   
182003  182003   13206  125506   
161023  161023  251013  251014   
188904  188904  287546  287547   
7638      7638   14915   11046   
129067  129067  207423  207424   
90619    90619  152036  152037   
108571  108571  178358  178359   
157023  157023  245610  245611   
87630    87630  147553  147554   
75347    75347  128971  128972   
7779      7779   15185   15186   
25284    25284   47140   47141   
37403    37403   68070   68071   
98181    98181  163194  163195   
140790  140790   25923  223671   
188993  188993  111923  287648   
195344  195344  295764   25025   
178507  178507

In [9]:
def tokenize_questions(df):
    question_1_tokenized = []
    question_2_tokenized = []
    for q in df.question1.tolist():
            question_1_tokenized.append([stemmer.stem(i.lower()) for i in words.findall(q) if i not in stopword])
    for q in df.question2.tolist():
        question_2_tokenized.append([stemmer.stem(i.lower()) for i in words.findall(q) if i not in stopword])
    df["Question_1_tok"] = question_1_tokenized
    df["Question_2_tok"] = question_2_tokenized
    return df

In [10]:
def train_dictionary(df):
    questions_tokenized = df.Question_1_tok.tolist() + df.Question_2_tok.tolist()
    #print(questions_tokenized)
    dictionary = corpora.Dictionary(questions_tokenized)
    #print(dictionary)
    dictionary.filter_extremes(no_below=5, no_above=0.8)
    dictionary.compactify()
    return dictionary

In [11]:
#print(df_train)
df_train = tokenize_questions(df_train)
#print("*********************************************")
#print(df_train)
dictionary = train_dictionary(df_train)
print ("No of words in the dictionary = %s" %len(dictionary.token2id))

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


No of words in the dictionary = 15140


In [12]:
df_test = tokenize_questions(df_test)
def get_vectors(df, dictionary):
    question1_vec = [dictionary.doc2bow(text) for text in df.Question_1_tok.tolist()]
    question2_vec = [dictionary.doc2bow(text) for text in df.Question_2_tok.tolist()]
    question1_csc = gensim.matutils.corpus2csc(question1_vec, num_terms=len(dictionary.token2id))
    question2_csc = gensim.matutils.corpus2csc(question2_vec, num_terms=len(dictionary.token2id))
    return question1_csc.transpose(),question2_csc.transpose()

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [13]:
q1_csc, q2_csc = get_vectors(df_train, dictionary)

print (q1_csc.shape)
print (q2_csc.shape)

(195999, 15140)
(195999, 15140)


In [14]:
from sklearn.metrics.pairwise import cosine_similarity as cs
from sklearn.metrics.pairwise import manhattan_distances as md
from sklearn.metrics.pairwise import euclidean_distances as ed
from sklearn.metrics import jaccard_similarity_score as jsc
from sklearn.neighbors import DistanceMetric
from sklearn.preprocessing import MinMaxScaler

minkowski_dis = DistanceMetric.get_metric('minkowski')
mms_scale_man = MinMaxScaler()
mms_scale_euc = MinMaxScaler()
mms_scale_mink = MinMaxScaler()

In [15]:
def get_similarity_values(q1_csc, q2_csc):
    cosine_sim = []
    manhattan_dis = []
    eucledian_dis = []
    jaccard_dis = []
    minkowsk_dis = []
    for i,j in zip(q1_csc, q2_csc):
        sim = cs(i,j)
        cosine_sim.append(sim[0][0])
        sim = md(i,j)
        manhattan_dis.append(sim[0][0])
        sim = ed(i,j)
        eucledian_dis.append(sim[0][0])
        i_ = i.toarray()
        j_ = j.toarray()
        try:
            sim = jsc(i_,j_)
            jaccard_dis.append(sim)
        except:
            jaccard_dis.append(0)
        sim = minkowski_dis.pairwise(i_,j_)
        minkowsk_dis.append(sim[0][0]) 
    return cosine_sim, manhattan_dis, eucledian_dis, jaccard_dis, minkowsk_dis    

In [16]:
cosine_sim, manhattan_dis, eucledian_dis, jaccard_dis, minkowsk_dis = get_similarity_values(q1_csc, q2_csc)
print ("cosine_sim sample= \n", cosine_sim[0:2])
print ("manhattan_dis sample = \n", manhattan_dis[0:2])
print ("eucledian_dis sample = \n", eucledian_dis[0:2])
print ("jaccard_dis sample = \n", jaccard_dis[0:2])
print ("minkowsk_dis sample = \n", minkowsk_dis[0:2])

('cosine_sim sample= \n', [0.9999999999999999, 0.6324555320336759])
('manhattan_dis sample = \n', [0.0, 7.0])
('eucledian_dis sample = \n', [0.0, 2.6457513110645907])
('jaccard_dis sample = \n', [1.0, 0])
('minkowsk_dis sample = \n', [0.0, 2.6457513110645907])


In [17]:
eucledian_dis_array = np.array(eucledian_dis).reshape(-1,1)
manhattan_dis_array = np.array(manhattan_dis).reshape(-1,1)
minkowsk_dis_array = np.array(minkowsk_dis).reshape(-1,1)

In [18]:
manhattan_dis_array = mms_scale_man.fit_transform(manhattan_dis_array)
eucledian_dis_array = mms_scale_euc.fit_transform(eucledian_dis_array)
minkowsk_dis_array = mms_scale_mink.fit_transform(minkowsk_dis_array)

eucledian_dis = eucledian_dis_array.flatten()
manhattan_dis = manhattan_dis_array.flatten()
minkowsk_dis = minkowsk_dis_array.flatten()

from sklearn.metrics import log_loss

def calculate_logloss(y_true, y_pred):
    loss_cal = log_loss(y_true, y_pred)
    return loss_cal

q1_csc_test, q2_csc_test = get_vectors(df_test, dictionary)
y_pred_cos, y_pred_man, y_pred_euc, y_pred_jac, y_pred_mink = get_similarity_values(q1_csc_test, q2_csc_test)
y_true = df_test.is_duplicate.tolist()

y_pred_man_array = mms_scale_man.transform(np.array(y_pred_man).reshape(-1,1))
y_pred_man = y_pred_man_array.flatten()

In [19]:
y_pred_euc_array = mms_scale_euc.transform(np.array(y_pred_euc).reshape(-1,1))
y_pred_euc = y_pred_euc_array.flatten()

y_pred_mink_array = mms_scale_mink.transform(np.array(y_pred_mink).reshape(-1,1))
y_pred_mink = y_pred_mink_array.flatten()

logloss = calculate_logloss(y_true, y_pred_cos)
print ("The calculated log loss value on the test set for cosine sim is = %f" %logloss)

logloss = calculate_logloss(y_true, y_pred_man)
print ("The calculated log loss value on the test set for manhattan sim is = %f" %logloss)

logloss = calculate_logloss(y_true, y_pred_euc)
print ("The calculated log loss value on the test set for euclidean sim is = %f" %logloss)

logloss = calculate_logloss(y_true, y_pred_jac)
print ("The calculated log loss value on the test set for jaccard sim is = %f" %logloss)

logloss = calculate_logloss(y_true, y_pred_mink)
print ("The calculated log loss value on the test set for minkowski sim is = %f" %logloss)

from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR

X_train = pd.DataFrame({"cos" : cosine_sim, "man" : manhattan_dis, "euc" : eucledian_dis, "jac" : jaccard_dis, "min" : minkowsk_dis})
y_train = df_train.is_duplicate

X_test = pd.DataFrame({"cos" : y_pred_cos, "man" : y_pred_man, "euc" : y_pred_euc, "jac" : y_pred_jac, "min" : y_pred_mink})
y_test = y_true

rfr = RandomForestRegressor()
rfr.fit(X_train, y_train)

The calculated log loss value on the test set for cosine sim is = 1.207097
The calculated log loss value on the test set for manhattan sim is = 2.619389
The calculated log loss value on the test set for euclidean sim is = 2.246408
The calculated log loss value on the test set for jaccard sim is = 3.066721
The calculated log loss value on the test set for minkowski sim is = 2.246408


/usr/local/lib/python2.7/dist-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [20]:
svr = SVR()
svr.fit(X_train,y_train)




/usr/local/lib/python2.7/dist-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1,
  gamma='auto_deprecated', kernel='rbf', max_iter=-1, shrinking=True,
  tol=0.001, verbose=False)

In [21]:


y_rfr_predicted = rfr.predict(X_test)
y_svr_predicted = svr.predict(X_test)

logloss_rfr = calculate_logloss(y_test, y_rfr_predicted)
logloss_svr = calculate_logloss(y_test, y_svr_predicted)

print ("The calculated log loss value on the test set using RFR is = %f" %logloss_rfr)
print ("The calculated log loss value on the test set using SVR is = %f" %logloss_svr)

The calculated log loss value on the test set using RFR is = 0.614497
The calculated log loss value on the test set using SVR is = 0.692607


In [22]:
y_svr_predicted

array([0.10226262, 0.5611114 , 0.90028825, ..., 0.10620058, 0.09285564,
       0.1006868 ])

In [23]:
X_test.head()

,cos,euc,jac,man,min
0,0.182574,0.137073,0.000000,0.050725,0.137073
1,0.866025,0.045691,0.750000,0.007246,0.045691
2,1.000000,0.000000,1.000000,0.000000,0.000000
3,0.000000,0.188390,0.000000,0.123188,0.188390
4,0.094491,0.204337,0.047619,0.144928,0.204337


In [24]:
df_test.head(1)

,id,qid1,qid2,question1,question2,is_duplicate,Question_1_tok,Question_2_tok
84010,84010,142096,142097,What should we answer when a HR questions why ...,Why should I I hire you?,1,"[what, answer, hr, question, hire]","[whi, i, i, hire]"


In [25]:
X_test.head(1)

,cos,euc,jac,man,min
0,0.182574,0.137073,0.0,0.050725,0.137073


In [26]:
y_svr_predicted[0]

0.10226261628016142

In [27]:
y_rfr_predicted
len(y_rfr_predicted)
print(y_rfr_predicted)

[0.         0.43119001 0.72456825 ... 0.53291141 0.         0.        ]


In [28]:
y_test
print(len(y_test))
print(y_test)

4000
[1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0

In [29]:
y_rfr_predicted=np.around(y_rfr_predicted)
print(y_rfr_predicted)

[0. 0. 1. ... 1. 0. 0.]


In [30]:
q=(y_test == y_rfr_predicted).sum()
print((y_test == y_rfr_predicted).sum())
print(q*100/len(y_test))

2778
69


In [31]:
print(len(y_test))
print(y_test)

4000
[1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0

In [32]:
y_svr_predicted=np.around(y_svr_predicted)
len(y_svr_predicted)
print(y_svr_predicted)

[0. 1. 1. ... 0. 0. 0.]


In [33]:
t=(y_test == y_svr_predicted).sum()
print((y_test == y_svr_predicted).sum())
print(t*100/len(y_test))

2544
63


In [34]:
import pickle


In [36]:
filename_svm = 'svm_models2la.sav'
filename_rgr = 'rgr_model2la.sav'

In [37]:
pickle.dump(svr,open(filename_svm,'wb'))

In [38]:
pickle.dump(rfr,open(filename_rgr,'wb'))